In [1]:
# Cell 1: Setup imports
import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Thêm đường dẫn gốc của dự án vào sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import TradingEngine và components
from src.core.trading_engine import TradingEngine
from src.strategies.sma_crossover import SMACrossoverStrategy
from src.strategies.rsi_strategy import RSIStrategy
from src.strategies.macd_strategy import MACDStrategy
from src.utils.config_manager import ConfigManager
from src.data.data_manager import DataManager

# Import visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

print("Import thành công!")

C:\Users\huynh.th\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\vnai\scope\profile.py:562: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Import thành công!


In [2]:
# Cell 2: Khởi tạo TradingEngine
config_path = "../../config/config.yaml"
engine = TradingEngine(config_path)

print(f"TradingEngine đã được khởi tạo với vốn ban đầu: ${engine.cash:,.2f}")
print(f"Config loaded: {engine.config.get('trading.symbols')}")
print(f"Config loaded: {engine.config.get('data.source')}")

2025-08-01 13:43:21.688 | INFO     | src.utils.config_manager:__init__:31 - Configuration loaded from ../../config/config.yaml
2025-08-01 13:43:21.691 | INFO     | src.risk.risk_manager:__init__:49 - Risk manager initialized
2025-08-01 13:43:21.691 | INFO     | src.core.trading_engine:__init__:77 - Trading engine initialized with $100,000.00 initial capital


TradingEngine đã được khởi tạo với vốn ban đầu: $100,000.00
Config loaded: ['Bitstamp:BTCUSD']
Config loaded: tradingview


In [3]:
# Cell 3: Chuẩn bị data
# Sử dụng data có sẵn hoặc lấy từ DataManager
symbols = engine.config.get("trading.symbols", ["AAPL"])
start_date = engine.config.get("data.start_date", "2023-01-01")
end_date = engine.config.get("data.end_date", "2023-12-31")


# Lấy data từ DataManager
data_manager = DataManager(engine.config)
historical_data = data_manager.get_historical_data(
    symbols=symbols,
    start_date=start_date,
    end_date=end_date,
    interval=engine.config.get("data.interval", "1d"),
    n_bars=engine.config.get("data.n_bars", 1000)
)

print(f"Lấy data cho symbols: {symbols}")
# print(f"Period: {start_date} đến {end_date}")
print(f"Data shape: {historical_data.shape}")
print(f"Data columns: {historical_data.columns.tolist()}")
print(
    f"Date range: {historical_data.index.min()} \
        đến {historical_data.index.max()} \
            interval: {engine.config.get('data.interval', '1d')}"
)

2025-08-01 13:43:23.403 | INFO     | src.data.data_manager:get_historical_data:60 - Fetching historical OHLCV data for ['Bitstamp:BTCUSD'] from 2023-01-01 to 2025-07-31
2025-08-01 13:43:24.015 | INFO     | src.data.data_manager:_fetch_tradingview_ohlcv_data:150 - Successfully fetched OHLCV data for 1 symbols
2025-08-01 13:43:24.016 | INFO     | src.data.data_manager:get_historical_data:93 - OHLCV data cached successfully


Lấy data cho symbols: ['Bitstamp:BTCUSD']
Data shape: (100, 5)
Data columns: [('Bitstamp:BTCUSD', 'open'), ('Bitstamp:BTCUSD', 'high'), ('Bitstamp:BTCUSD', 'low'), ('Bitstamp:BTCUSD', 'close'), ('Bitstamp:BTCUSD', 'volume')]
Date range: 2025-07-15 23:00:00         đến 2025-08-01 11:00:00             interval: 4h


In [4]:
symbol = symbols[0]
print(f'Visualize data {symbol}')
# Visualize
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)

fig.add_trace(go.Candlestick(x=historical_data[symbol].index,
                    open=historical_data[symbol].open,
                    high=historical_data[symbol].high,
                    low=historical_data[symbol].low,
                    close=historical_data[symbol].close,
                    ), row=1, col=1)
set_ylim = (historical_data[symbol].low.min(), historical_data[symbol].high.max() * 1.05)

fig.add_trace(go.Bar(x=historical_data[symbol].index,
                    y=historical_data[symbol].volume,
                    ), row=2, col=1)

fig.update_layout(title=f'{symbol}',
                  yaxis_range=(set_ylim[0], set_ylim[1]),
                  xaxis_title='Date',
                  yaxis_title='Price')
fig.show()

Visualize data Bitstamp:BTCUSD


In [4]:
# Cell 4: Thêm strategies
# SMA Crossover Strategy
sma_strategy = SMACrossoverStrategy({
    "short_window": 10,
    "long_window": 30,
    "name": "SMA_Crossover"
})
engine.add_strategy(sma_strategy)

# RSI Strategy
rsi_strategy = RSIStrategy({
    "period": 14,
    "oversold": 30,
    "overbought": 70,
    "name": "RSI_Strategy"
})
engine.add_strategy(rsi_strategy)

# MACD Strategy
macd_strategy = MACDStrategy({
    "fast_period": 12,
    "slow_period": 26,
    "signal_period": 9,
    "name": "MACD_Strategy"
})
engine.add_strategy(macd_strategy)

print(f"Đã thêm {len(engine.strategies)} strategies:")
for name, strategy in engine.strategies.items():
    print(f"  - {name}: {strategy.__class__.__name__}")

2025-08-01 13:43:24.815 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: sma_crossover
2025-08-01 13:43:24.816 | INFO     | src.strategies.sma_crossover:__init__:48 - SMA Crossover Strategy initialized: 10/30
2025-08-01 13:43:24.816 | INFO     | src.core.trading_engine:add_strategy:82 - Added strategy: sma_crossover
2025-08-01 13:43:24.817 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: rsi
2025-08-01 13:43:24.817 | INFO     | src.strategies.rsi_strategy:__init__:56 - RSI Strategy initialized: period=14, overbought=70, oversold=30
2025-08-01 13:43:24.818 | INFO     | src.core.trading_engine:add_strategy:82 - Added strategy: rsi
2025-08-01 13:43:24.818 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: macd
2025-08-01 13:43:24.818 | INFO     | src.strategies.macd_strategy:__init__:54 - MACD Strategy initialized: fast=12, slow=26, signal=9
2025-08-01 13:43:24.819 | INFO     | src.core.trading_engine:add

Đã thêm 3 strategies:
  - sma_crossover: SMACrossoverStrategy
  - rsi: RSIStrategy
  - macd: MACDStrategy


In [5]:
# Cell 5: Chạy backtest
print("Bắt đầu chạy backtest...")

# Chạy chiến lược
print('SMA Crossover:', engine.config.get('strategies.sma_crossover.enabled', False))
print('RSI:', engine.config.get('strategies.rsi.enabled', False))
print('MACD:', engine.config.get('strategies.macd.enabled', False), '\n')

# Chạy backtest với chiến lược đã chọn
engine.run_backtest(start_date, end_date)

2025-08-01 13:43:28.439 | INFO     | src.core.trading_engine:run_backtest:86 - Starting backtest from 2023-01-01 to 2025-07-31
2025-08-01 13:43:28.440 | INFO     | src.data.data_manager:get_historical_data_standardized:609 - Fetching standardized OHLCV data for ['Bitstamp:BTCUSD'] from 2023-01-01 to 2025-07-31
2025-08-01 13:43:28.441 | INFO     | src.data.data_manager:get_historical_data:60 - Fetching historical OHLCV data for ['Bitstamp:BTCUSD'] from 2023-01-01 to 2025-07-31


Bắt đầu chạy backtest...
SMA Crossover: True
RSI: False
MACD: False 



2025-08-01 13:43:29.377 | INFO     | src.data.data_manager:_fetch_tradingview_ohlcv_data:150 - Successfully fetched OHLCV data for 1 symbols
2025-08-01 13:43:29.379 | INFO     | src.data.data_manager:get_historical_data:93 - OHLCV data cached successfully
2025-08-01 13:43:29.381 | INFO     | src.data.data_manager:ensure_ohlcv_format:524 - Data formatted for Bitstamp:BTCUSD: (1000, 5)
2025-08-01 13:43:29.382 | INFO     | src.data.data_manager:get_historical_data_standardized:642 - Successfully standardized OHLCV data for 1 symbols
2025-08-01 13:43:29.386 | INFO     | src.core.trading_engine:run_backtest:107 - Loaded 1000 data points for [('Bitstamp:BTCUSD', 'open'), ('Bitstamp:BTCUSD', 'high'), ('Bitstamp:BTCUSD', 'low'), ('Bitstamp:BTCUSD', 'close'), ('Bitstamp:BTCUSD', 'volume')]
2025-08-01 13:43:29.394 | INFO     | src.strategies.sma_crossover:_generate_signal_for_symbol:109 - Golden cross detected for Bitstamp:BTCUSD: 96455.20 > 96324.63
2025-08-01 13:43:29.395 | INFO     | src.core

In [6]:
# Cell 6: Lấy kết quả backtest
portfolio_summary = engine.get_portfolio_summary()

print("=== KẾT QUẢ BACKTEST ===")
print(f"Initial Capital: ${portfolio_summary['initial_capital']:,.2f}")
print(f"Final Portfolio Value: ${portfolio_summary['total_value']:,.2f}")
print(f"Total Return: {portfolio_summary['total_return']:.2%}")
print(f"Annualized Return: {portfolio_summary['annualized_return']:.2%}")
print(f"Sharpe Ratio: {portfolio_summary['sharpe_ratio']:.2f}")
print(f"Max Drawdown: {portfolio_summary['max_drawdown']:.2%}")
print(f"Win Rate: {portfolio_summary['win_rate']:.2%}")
print(f"Total Trades: {portfolio_summary['total_trades']}")

=== KẾT QUẢ BACKTEST ===
Initial Capital: $100,000.00
Final Portfolio Value: $101,267.31
Total Return: 1.27%
Annualized Return: 0.32%
Sharpe Ratio: 0.15
Max Drawdown: -4.59%
Win Rate: 58.33%
Total Trades: 48


In [7]:
# Cell 7: Visualize portfolio performance
def plot_portfolio_performance(engine):
    """Plot portfolio performance"""
    portfolio_history = engine.portfolio_history
    
    if not portfolio_history:
        print("Không có dữ liệu portfolio history")
        return
    
    df = pd.DataFrame(portfolio_history)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    
    # Create subplots
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=('Portfolio Value', 'Daily Returns', 'Cumulative Returns', 
                       'Drawdown', 'Cash vs Positions', 'Trade Distribution'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Portfolio Value
    fig.add_trace(
        go.Scatter(x=df.index, y=df['total_value'], 
                  mode='lines', name='Portfolio Value'),
        row=1, col=1
    )
    
    # Daily Returns
    daily_returns = df['total_value'].pct_change()
    fig.add_trace(
        go.Scatter(x=df.index, y=daily_returns, 
                  mode='lines', name='Daily Returns'),
        row=1, col=2
    )
    
    # Cumulative Returns
    cumulative_returns = (1 + daily_returns).cumprod()
    fig.add_trace(
        go.Scatter(x=df.index, y=cumulative_returns, 
                  mode='lines', name='Cumulative Returns'),
        row=2, col=1
    )
    
    # Drawdown
    running_max = df['total_value'].expanding().max()
    drawdown = (df['total_value'] - running_max) / running_max
    fig.add_trace(
        go.Scatter(x=df.index, y=drawdown, 
                  mode='lines', name='Drawdown', fill='tonexty'),
        row=2, col=2
    )
    
    # Cash vs Positions
    fig.add_trace(
        go.Scatter(x=df.index, y=df['cash'], 
                  mode='lines', name='Cash'),
        row=3, col=1
    )
    fig.add_trace(
        go.Scatter(x=df.index, y=df['total_value'] - df['cash'], 
                  mode='lines', name='Positions'),
        row=3, col=1
    )
    
    # Trade Distribution (if available)
    if hasattr(engine, 'trades') and engine.trades:
        trade_returns = [trade.pnl for trade in engine.trades]
        fig.add_trace(
            go.Histogram(x=trade_returns, name='Trade Returns'),
            row=3, col=2
        )
    
    fig.update_layout(height=900, title_text="Portfolio Performance Analysis")
    fig.show()

# Chạy visualization
plot_portfolio_performance(engine)

In [8]:
# Cell 8: Hiển thị chi tiết trades
if hasattr(engine, 'trades') and engine.trades:
    print(f"\n=== CHI TIẾT TRADES ({len(engine.trades)} trades) ===")
    
    trades_df = pd.DataFrame([
        {
            'Symbol': trade.symbol,
            'Side': trade.side,
            'Quantity': trade.quantity,
            'Price': trade.price,
            'Timestamp': trade.timestamp,
            'Commission': trade.commission,
            'Strategy': trade.strategy
        }
        for trade in engine.trades
    ])
    
    print(trades_df.head(10))
    
    # Trade statistics
    print(f"\n=== TRADE STATISTICS ===")
    print(f"Total Trades: {len(engine.trades)}")
    print(f"Buy Trades: {len(trades_df[trades_df['Side'] == 'buy'])}")
    print(f"Sell Trades: {len(trades_df[trades_df['Side'] == 'sell'])}")
    print(f"Average Trade Size: {trades_df['Quantity'].mean():.2f}")
    print(f"Total Commission: ${trades_df['Commission'].sum():.2f}")
else:
    print("Không có trades nào được thực hiện")


=== CHI TIẾT TRADES (48 trades) ===
            Symbol  Side  Quantity    Price           Timestamp  Commission  \
0  Bitstamp:BTCUSD   buy  0.206460  96871.0 2025-02-20 19:00:00   20.000000   
1  Bitstamp:BTCUSD  sell  0.206460  96448.0 2025-02-23 07:00:00   19.912667   
2  Bitstamp:BTCUSD   buy  0.232049  86079.0 2025-03-02 15:00:00   19.974551   
3  Bitstamp:BTCUSD  sell  0.232049  86765.0 2025-03-05 07:00:00   20.133737   
4  Bitstamp:BTCUSD   buy  0.218411  91563.0 2025-03-06 07:00:00   19.998366   
5  Bitstamp:BTCUSD  sell  0.218411  86179.0 2025-03-08 11:00:00   18.822442   
6  Bitstamp:BTCUSD   buy  0.237636  83133.0 2025-03-13 11:00:00   19.755417   
7  Bitstamp:BTCUSD  sell  0.237636  81606.0 2025-03-18 19:00:00   19.392547   
8  Bitstamp:BTCUSD   buy  0.229030  85906.0 2025-03-20 07:00:00   19.675014   
9  Bitstamp:BTCUSD  sell  0.229030  84107.0 2025-03-22 19:00:00   19.262989   

        Strategy  
0  sma_crossover  
1  sma_crossover  
2  sma_crossover  
3  sma_crossover 

In [9]:
# Cell 9: So sánh performance của các strategies
def compare_strategies(engine):
    """Compare performance of different strategies"""
    if not hasattr(engine, 'strategies') or not engine.strategies:
        print("Không có strategies để so sánh")
        return
    
    strategy_results = {}
    
    for name, strategy in engine.strategies.items():
        # Calculate basic metrics for each strategy
        # This is a simplified version - you might want to run separate backtests
        strategy_results[name] = {
            'name': name,
            'type': strategy.__class__.__name__,
            'parameters': strategy.get_summary()
        }
    
    # Display strategy comparison
    print("=== STRATEGY COMPARISON ===")
    for name, result in strategy_results.items():
        print(f"\nStrategy: {name}")
        print(f"Type: {result['type']}")
        print(f"Parameters: {result['parameters']}")

compare_strategies(engine)

=== STRATEGY COMPARISON ===

Strategy: sma_crossover
Type: SMACrossoverStrategy
Parameters: {'name': 'sma_crossover', 'strategy_type': 'SMA Crossover', 'short_window': 10, 'long_window': 30, 'description': 'SMA Crossover with 10/30 periods'}

Strategy: rsi
Type: RSIStrategy
Parameters: {'name': 'rsi', 'strategy_type': 'RSI Strategy', 'period': 14, 'overbought_threshold': 70, 'oversold_threshold': 30, 'confirmation_period': 2, 'description': 'RSI Strategy with 14 period, 30/70 levels'}

Strategy: macd
Type: MACDStrategy
Parameters: {'name': 'macd', 'strategy_type': 'MACD Strategy', 'fast_period': 12, 'slow_period': 26, 'signal_period': 9, 'histogram_threshold': 0.0, 'confirmation_period': 1, 'description': 'MACD Strategy with 12/26/9 periods'}
